In [1]:
import random
import json
import pickle
import importlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import scipy.spatial
import sklearn
from sklearn import preprocessing,decomposition,linear_model

import progress
import jsdump
import latlon
import polygons
import polygongraph
import footprint_kml_poly_extractor
import FloatColorMapper

In [2]:
# set the (longitude, latitude) center for for the distance approximation calculator
# use a point near the center of the place you're at
# the current coordinates are for Champaign, IL
CENTER_X = -88.2434
CENTER_Y = 40.1164

latlon.set_center(CENTER_X,CENTER_Y)

In [3]:
# get a list of MultiPolygon objects representing house footprints
house_polys = footprint_kml_poly_extractor.extract()

In [4]:
# read tax data that was extracted from PropertyShark
tax_frame = pd.read_csv('PropertyShark/tax_all.csv')
# get averages out of places with taxes found
tax_average = tax_frame[tax_frame['taxes_found']==1].mean(0)
tax_average

address_found          1.000000
taxes_found            1.000000
tax_year            2019.000000
tax_amount          6970.514557
market_value      235439.533823
land_value         55474.155970
building_value    179965.377853
dtype: float64

In [5]:
# read champaign-addresses.csv
addr_frame = pd.read_csv('champaign-addresses.csv')
# get rid of inactive and non-residential addresses
addr_frame = addr_frame[list(addr_frame['Status']=='Active')]
addr_frame = addr_frame[list(addr_frame['Residential']=='Y')]
addr_frame.reset_index(drop=True, inplace=True)
# create 'address' column to merge with tax_frame
addr_frame['address'] = addr_frame['StreetAddress']

# merge with tax_frame
addr_frame = addr_frame.merge(tax_frame,on='address')
print(addr_frame.shape)

# create addr_pts: list of x,y coordinates for the addresses
addr_pts = addr_frame[['X','Y']].values

# save addr_frame
addr_frame.to_csv('addresses_with_tax.csv')

/home/xanthi/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (17,28,31,32,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(45674, 53)


In [6]:
importlib.reload(polygons)
putted_indices = polygons.put_points_into_polys(addr_pts, house_polys, updateEvery=10000)

START (45674 total, updates every 10000)
10000 / 45674 = 0.219 (2.20 s)
20000 / 45674 = 0.438 (5.23 s)
30000 / 45674 = 0.657 (8.01 s)
40000 / 45674 = 0.876 (10.30 s)
45674 / 45674 = 1.000 (11.31 s)


In [7]:
class House:
    """
    An object that contains data for a single residential building.
    Stores a list of addresses that correspond to this building.
    The address list could have size 1 (if it's a single-family house) 
        or be very long (if it's an apartment)
    """
    def __init__(self, footprint):
        """
        Create a House object with a given footprint.
        The list of addresses starts out empty - use addPlace to add an address
         - footprint: MultiPolygon
        """
        # footprint (MultiPolygon): house footprint (MultiPolygon)
        self.footprint = footprint
        # centroid (np.ndarray of shape (2,)): house centroid in the form [x,y] 
        self.centroid = footprint.centroid()
        # numAddr (int): number of addresses corresponding to this house
        self.numAddr = 0
        # nClosest ([int]): contains a list of the nClosest_n closest houses
        # nClosest_n (int): the size of nClosest
        # The closest is always the house itself, so we initialize nClosest with [0]
        # and nClosest_n with 1.
        # Given a list of houses, this can be updated with House.setNClosest. 
        self.nClosest_n = 1
        self.nClosest = [0.]
        # addrs ({str}): the set of unique addresses that correspond to this house
        # does NOT include subaddresses (like Apt 23)
        self.addrs = set()
        # tax_data (DataFrame): a dataframe including the tax frame rows
        # for the addresses corresponding to this house
        self.tax_data = pd.DataFrame(columns=tax_frame.columns)
        # data_list ([Series]): list of dataframe rows added by self.addPlace
        self.data_list = []
        # features (np.ndarray with 1 dimension): features for the house
        # Starts out with a single garbage feature (can be updated later by finalize_features)
        self.features = np.random.rand(1) # features: starts out with garbage data
    def addPlace(self,row):
        """
        Adds a row (from addr_frame) to the list of addresses for this house
        """
        addr = row['address']
        self.numAddr += 1
        self.data_list.append(row)
        if addr not in self.addrs:
            self.addrs.add(addr)
            if row['taxes_found']:
                self.tax_data.loc[len(self.tax_data)] = row[tax_frame.columns]
    @staticmethod
    def setNClosest(houses, nClosest_n=10):
        """
        Given a list of houses, sets the nClosest and nClosest_n attribute for the houses
        """
        house_cents = np.stack([h.centroid for h in houses],0)
        for house_i,house in enumerate(houses):
            dists = latlon.approx_dist(house_cents[house_i],house_cents)
            dists.sort()
            house.nClosest_n = nClosest_n
            house.nClosest = dists[:house.nClosest_n]

print("Creating houses...")
houses = [House(fp) for fp in house_polys]
print("Putting addresses into houses...")
prog = progress.Progress(len(putted_indices),5000)
for putted_i,putted in enumerate(putted_indices):
    prog.update()
    if putted == -1:
        continue
    houses[putted].addPlace(addr_frame.iloc[putted_i])

print("Removing useless houses...")
houses = [h for h in houses if h.numAddr>0]

print("Calculating closest houses...")
House.setNClosest(houses)
print("Done!")

Creating houses...
Putting addresses into houses...
START (45674 total, updates every 5000)
5000 / 45674 = 0.109 (8.03 s)
10000 / 45674 = 0.219 (13.74 s)
15000 / 45674 = 0.328 (20.70 s)
20000 / 45674 = 0.438 (26.94 s)
25000 / 45674 = 0.547 (33.83 s)
30000 / 45674 = 0.657 (43.47 s)
35000 / 45674 = 0.766 (47.04 s)
40000 / 45674 = 0.876 (49.27 s)
45000 / 45674 = 0.985 (50.55 s)
45674 / 45674 = 1.000 (50.60 s)
Removing useless houses...
Calculating closest houses...
Done!


In [8]:
# a list of feature names (there are 8 features)
FEATURE_NAMES = [
    'area',
    'dist_closest',
    'dist_closest2',
    'num_addresses',
    'tax_amount',
    'land_value',
    'building_value',
    'land_value_per_area'
]

In [9]:
def finalize_features(houses, num_reduced_features=2):
    """Given a list of houses, finalizes their 'features' attribute
    relevant features:
     - 0 | house area (house.footprint.totalArea)
     - 1 | distance to closest house (house.nClosest[1])
     - 2 | distance to 2nd closest house (house.nClosest[2])
     - 3 | number of residential addresses (house.numAddr)
     - 4 | average tax amount for addresses (tax_amount)
     - 5 | average land value for addresses (land_value)
     - 6 | average building value for addresses (building_value)
     - 7 | land value per square unit (land_value/house_area)
    """
    num_features = 8
    features = np.zeros((len(houses),num_features))
    # generate features
    for house_i,house in enumerate(houses):
        f_area = house.footprint.totalArea
        f_dist1 = house.nClosest[1]
        f_dist2 = house.nClosest[2]
        f_numaddr = house.numAddr
        if len(house.tax_data):
            tdmean = house.tax_data.mean(0)
        else:
            tdmean = tax_average
        def gen_tdmean_attr(attr):
            return tdmean[attr] if tdmean[attr] != 0 else tax_average[attr]
        f_tax_amount = gen_tdmean_attr('tax_amount')
        f_land_value = gen_tdmean_attr('land_value')
        f_building_value = gen_tdmean_attr('building_value')
        f_sqfoot_value = f_land_value/f_area
        features[house_i,:] = np.array([f_area,f_dist1,f_dist2,f_numaddr,
                                        f_tax_amount,f_land_value,f_building_value,f_sqfoot_value])
        for zz in range(8):
            if features[house_i][zz] == 0:
                print(house_i,zz)
    # normalize features using log
    features_log = np.log2(features)
    features_final = features_log
    # send features back to house objects
    for house_i,house in enumerate(houses):
        house.features = features_final[house_i,:]

finalize_features(houses)

In [10]:
"""
DISTANCE SUPERVISION MODEL
This model attempts to learn weights for the 8 house features,
by training a logistic regression classifier to attempt to classify between two houses being
close together (distance less than DIST_MAX), or far apart (distance greater than DIST_MAX)
The weights learned by this logistic regression are then used as weights for the 8 house features
"""

### calculate feature importance using distance supervision ###
DIST_MAX = .3  # in km
NUM_TRAIN = 5000  # this better be a multiple of 2

# gather training data
N_HOUSES = len(houses)
NUM_TRAIN_EACH = NUM_TRAIN/2
ds_close_points = []
ds_far_points = []

while len(ds_close_points)+len(ds_far_points) < NUM_TRAIN:
    hi,hj = tuple(random.sample(range(N_HOUSES),2))
    d = latlon.approx_dist(houses[hi].centroid, houses[hj].centroid)
    datapoint = (houses[hi].features, houses[hj].features)
    if d <= DIST_MAX and len(ds_close_points) < NUM_TRAIN/2:
        ds_close_points.append(datapoint)
    elif d > DIST_MAX and len(ds_far_points) < NUM_TRAIN/2:
        ds_far_points.append(datapoint)

def get_feature_vec(hf1, hf2):
    """given two house features, compute the distance feature to be used for distance supervision
    hf1 (np.ndarray of size 8): house features for house 1
    hf2 (np.ndarray of size 8): house features for house 2
    """
    return np.abs(hf1-hf2)

def stacked_feature_vecs(ds_points):
    """converts ds_close_points or ds_far_points into a stacked tuple of 
        logistic regression feature vectors
    ds_points ([(np.ndarray, np.ndarray)]): either ds_close_points or ds_far_points
    """
    return np.stack([get_feature_vec(*p) for p in ds_points])

# training data for the logistic regression
y_train = np.concatenate((
    np.zeros(len(ds_close_points)),
    np.ones(len(ds_far_points))
))
x_train = np.concatenate((
    stacked_feature_vecs(ds_close_points),
    stacked_feature_vecs(ds_far_points)
))

In [11]:
ds_model = linear_model.LogisticRegression(fit_intercept=True)
ds_model.fit(x_train, y_train)

LogisticRegression()

In [12]:
# print out some info about the logistic regression
ds_model.predict_proba(x_train[:10])
# ds_model.score(x_train, y_train)
print(ds_model.coef_,ds_model.intercept_)
# generic saved weights: [ 0.34041744, 0.92774432, 0.09131713, -0.10435152, -0.17135961, 0.98659969, 0.57859899, 0.05270441]

[[ 0.31096528  1.07109025  0.01828399 -0.07759281 -0.21611248  1.14433986
   0.45912157  0.05385504]] [-1.60912928]


In [13]:
# load the road regions (generated from road_region_creator.ipynb)
road_poly_file = open('road_regions.json','r')
road_polys = polygons.PolyList.import_json(road_poly_file)

# create a polygon graph containing the road regions
importlib.reload(polygongraph)
house_cents = np.stack([h.centroid for h in houses],0)
graph = polygongraph.PolygonGraph(road_polys, house_cents, houses)
print(graph.N)

1055


In [14]:
importlib.reload(polygons)
def get_features(houses):
    """gets a stacked ndarray of features for a list of houses
     - houses: [House]
    output: np.ndarray
    """
    return np.stack([h.features for h in houses])

def dist_features(houses1, houses2, explain=False):
    """gets the dissimilarity measure between two lists of houses,
        to be used with AGNES
     - houses1 ([House])
     - houses2 ([House])
     - explain: if true, instead of returning a dissimilarity, 
         returns a string explaining why the dissimilarity is calculated to what it is
    """
    feat1 = get_features(houses1)
    feat2 = get_features(houses2)
    f1mean = feat1.mean(0)
    f2mean = feat2.mean(0)
    ds_fvec = get_feature_vec(f1mean, f2mean)
    
    # use the weights learned from the distance supervision model
    fweights = ds_model.coef_[0].copy()
    # weights below 0 are set to 0 (because negative weights don't make sense)
    fweights[fweights<0] = 0
    
    # explain the result
    if explain:
        v = ds_fvec*fweights
        n_feats = len(FEATURE_NAMES)
        res = []
        for i in range(n_feats):
            if fweights[i] != 0:
                res.append((v[i],FEATURE_NAMES[i]))
        res.sort(reverse=True)
        return ', '.join([t[1]+' '+str(t[0]) for t in res])
        
    # return the weighted sum
    return np.dot(ds_fvec, fweights)
    

def dist_func(node1, node2, explain=False):
    """Returns the dissimilarity measure between two nodes in a PolygonGraph
     - node1 (PolygonNode)
     - node2 (PolygonNode)
     - explain: whether or not to return a string explaning the result
    """
    # get the dissimilarity based on data features
    dist_f = dist_features(node1.data, node2.data, explain)
    # get the shared perimeter proportion between the 2 polygons
    perim_f = polygons.MultiPolygon.shared_perimeters(node1.poly,node2.poly) / min(node1.poly.perimeter(),node2.poly.perimeter())
    unperim_f = 1-perim_f
    if explain:
        if perim_f == 0:
            return 'These polygons don\'t even share an edge...'
        return 'Feature impact: %s\nPerimeter Modifier: %f' % (dist_f, unperim_f)
    return dist_f * (1+unperim_f)

# run AGNES
graph.agnes_cluster(dist_func, 1)
# number of clusters at the end
print(graph.agnes_min_clusters)

12


In [15]:
# set the number of clusters in the final result (based on maximum dissimilarity)
graph.agnes_set_num_clusters(max_dist=1)
print(graph.agnes_get_stats())

# store AGNES results into a js file
agnes_res = graph.agnes_get_result_polygons()
agnes_labels = graph.agnes_get_labelled_centroids()
dumper = jsdump.Dumper('agnes_road_regions.js')
dumper.dump('agnes_road_regions', polygons.PolyList.to_vertex_lists(agnes_res))
dumper.dump('agnes_centroid_labels', agnes_labels)
dumper.done()

Base polygons (1055): 0-1054
Merged polygons (696): 1055-1750
Active polygons (359)


In [16]:
# an example of using agnes_explain
print(graph.agnes_explain(1536))

(1536,1308) -> 1598
1536 <- (54,1535)



In [17]:
# an example of using agnes_explain_pair
print(graph.agnes_explain_pair(0,1))

Feature impact: dist_closest 0.31291258817612977, land_value 0.28166119813799795, building_value 0.09269085484718631, area 0.031519535586215106, dist_closest2 0.00827999724864551, land_value_per_area 0.00779680472829889
Perimeter Modifier: 0.653487


In [18]:
# this allows visualizing a colormap of specific features on the generated Google maps
importlib.reload(FloatColorMapper)
def gen_floatcolors_json():
    floatcolors_json = {}
    house_cents = [h.centroid for h in houses]
    for fi,fname in enumerate(FEATURE_NAMES):
        feats = [h.features[fi] for h in houses]
        floatcolors_json[fname] = FloatColorMapper.genJSON(
            house_cents, feats,
            norm_func=FloatColorMapper.normalize_minmax
        )
    return floatcolors_json
floatcolors_json = gen_floatcolors_json()
dumper = jsdump.Dumper('feature_color_map.js')
dumper.dump('feature_color_map', floatcolors_json)
dumper.done()

In [19]:
# agnes address list generator
# json output: [id: {addrs: [...], edges: [...]}]
# addresses are CompleteAddress
agnes_json = {}
for node_i in graph.agnes_get_result_node_idx():
    addrs = []
    for node_data in graph.agnes_nodes[node_i].data:
        addrs += [row['CompleteAddress'] for row in node_data.data_list]
    edges = [str(j) for j in graph.agnes_edges_for_node(node_i)]
    agnes_json[node_i] = {'addrs': addrs, 'edges': edges}

with open('agnes_result.json', 'w') as output:
    json.dump(agnes_json, output, indent=2)